In [1]:
!pip install spotipy
!pip install billboard.py
!pip install lyricsgenius
!pip install nltk

In [2]:
import pandas as pd
import numpy as np
import billboard as bb
import lyricsgenius
import nltk
from sklearn.feature_extraction.text import CountVectorizer
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
genius = lyricsgenius.Genius("JxIIf2vA02Oopvz2JRUbC6iysXRZnNEHN0j84a16uLsB8niafpsNZPR1SvcSxu8r")
genius.verbose = False # Turn off status messages
genius.remove_section_headers = True # Remove section headers (e.g. [Chorus]) from lyrics when searching
genius.skip_non_songs = False # Include hits thought to be non-songs (e.g. track lists)
genius.excluded_terms = ["(Remix)", "(Live)"] # Exclude songs with these words in their title

In [4]:
def create_songs_df(chart_name):
    top100 = bb.ChartData(chart_name)
    songs = top100.entries
    dataframe = {"Rank": [], "Title": [], "Artist": []}
    for song in songs:
        dataframe["Title"].append(song.title)
        dataframe["Artist"].append(song.artist)
        dataframe["Rank"].append(song.rank)
    df = pd.DataFrame(dataframe)
    return df

In [5]:
top100 = create_songs_df("hot-100")
country = create_songs_df("country-songs")
rock = create_songs_df("rock-songs")
pop = create_songs_df("pop-songs")
print(top100)

    Rank              Title                                            Artist
0      1            Circles                                       Post Malone
1      2  Someone You Loved                                     Lewis Capaldi
2      3       Good As Hell                                             Lizzo
3      4           Memories                                          Maroon 5
4      5            Roxanne                                    Arizona Zervas
..   ...                ...                                               ...
95    96    Hot Girl Summer  Megan Thee Stallion, Nicki Minaj & Ty Dolla $ign
96    97       Lonely Child                        YoungBoy Never Broke Again
97    98   Into The Unknown                               Panic! At The Disco
98    99      Show Yourself                   Idina Menzel & Evan Rachel Wood
99   100           Homesick                                        Kane Brown

[100 rows x 3 columns]


In [6]:
def try_load_lyrics(title,artist):
    try:
        return genius.search_song(title, artist).lyrics
    except:
        return "Failed to Find Lyrics"

In [7]:
def populate_chart_with_lyrics(chart):
    chart["lyrics"] = chart.apply(lambda x: try_load_lyrics(x["Title"],x["Artist"]), axis = 1)
    return chart

In [8]:
top100 = populate_chart_with_lyrics(top100)
display(top100)
len(top100[top100['lyrics'] == 'Failed to Find Lyrics'])

,Rank,Title,Artist,lyrics
0,1,Circles,Post Malone,"Oh, oh, oh\nOh, oh, oh\nOh, oh, oh, oh, oh\n..."
1,2,Someone You Loved,Lewis Capaldi,"I'm going under, and this time, I fear there's..."
2,3,Good As Hell,Lizzo,"I do my hair toss, check my nails\nBaby, how y..."
3,4,Memories,Maroon 5,Here's to the ones that we got\nCheers to the ...
4,5,Roxanne,Arizona Zervas,All for the 'Gram\nBitches love the 'Gram\nOh ...
...,...,...,...,...
95,96,Hot Girl Summer,"Megan Thee Stallion, Nicki Minaj & Ty Dolla $ign","Ransom - Lil Tecca\nBezerk - Big Sean, A$AP Fe..."
96,97,Lonely Child,YoungBoy Never Broke Again,"Pipe that shit up, TNT\nI just wanna be loved\..."
97,98,Into The Unknown,Panic! At The Disco,"Ah-ah, oh-oh\nAh-ah, oh-oh, oh-oh\nInto the un..."
98,99,Show Yourself,Idina Menzel & Evan Rachel Wood,Every inch of me is trembling\nBut not from th...


1

In [9]:
client_credentials_manager = SpotifyClientCredentials(client_id = "ee9a1f8e2f634959967d52ed9fad54de", client_secret = "934a6bbc90e74664b502638f6f5fc92b")
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [10]:
def get_spotify_popularity(title):
    title = title.replace("$","s")
    answer = sp.search(title, type="track", limit = 1)
    track = sp.track(answer["tracks"]["items"][0]["id"])
    return track["popularity"]

In [11]:
def populate_chart_with_spotify_popularity(chart):
    chart["spotify_popularity"] = chart.apply(lambda x: get_spotify_popularity(x["Title"]), axis = 1)
    return chart

In [12]:
top100 = populate_chart_with_spotify_popularity(top100)
display(top100)

,Rank,Title,Artist,lyrics,spotify_popularity
0,1,Circles,Post Malone,"Oh, oh, oh\nOh, oh, oh\nOh, oh, oh, oh, oh\n...",99
1,2,Someone You Loved,Lewis Capaldi,"I'm going under, and this time, I fear there's...",96
2,3,Good As Hell,Lizzo,"I do my hair toss, check my nails\nBaby, how y...",86
3,4,Memories,Maroon 5,Here's to the ones that we got\nCheers to the ...,100
4,5,Roxanne,Arizona Zervas,All for the 'Gram\nBitches love the 'Gram\nOh ...,97
...,...,...,...,...,...
95,96,Hot Girl Summer,"Megan Thee Stallion, Nicki Minaj & Ty Dolla $ign","Ransom - Lil Tecca\nBezerk - Big Sean, A$AP Fe...",83
96,97,Lonely Child,YoungBoy Never Broke Again,"Pipe that shit up, TNT\nI just wanna be loved\...",75
97,98,Into The Unknown,Panic! At The Disco,"Ah-ah, oh-oh\nAh-ah, oh-oh, oh-oh\nInto the un...",86
98,99,Show Yourself,Idina Menzel & Evan Rachel Wood,Every inch of me is trembling\nBut not from th...,82


In [13]:
def create_term_doc_matrix(chart):
    lyrics = chart["lyrics"].to_list()
    vec = CountVectorizer()
    X = vec.fit_transform(lyrics)
    termdocmatrix = pd.DataFrame(X.toarray(), columns=vec.get_feature_names(), index=chart.Title)
    return termdocmatrix

top100 = top100[top100.lyrics != "Failed to Find Lyrics"]
create_term_doc_matrix(top100)

,000,02,10,100,101,10k,11,12,12am,13,...,کردن,کردیم,کسی,کلی,کمگ,کنار,کنیم,که,گرفتن,یه
Title,,,,,,,,,,,,,,,,,,,,,
Circles,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Someone You Loved,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Good As Hell,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Memories,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Roxanne,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Hot Girl Summer,0,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Lonely Child,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Into The Unknown,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
def create_tfidf(chart):
    lyrics = chart["lyrics"].to_list()
    vectorizer = TfidfVectorizer()
    doc_vec = vectorizer.fit_transform(lyrics)
    tfidf = pd.DataFrame(doc_vec.toarray().transpose(), index=vectorizer.get_feature_names())
    tfidf.columns = chart["Title"]
    return tfidf

tfidf = create_tfidf(top100)
tfidf

Title,Circles,Someone You Loved,Good As Hell,Memories,Roxanne,Lose You To Love Me,Senorita,"10,000 Hours",Truth Hurts,No Guidance,...,Make No Sense,Good Vibes,La Cancion,Easy,Cuban Links,Hot Girl Summer,Lonely Child,Into The Unknown,Show Yourself,Homesick
000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.008303,0.000000,0.0,0.0,0.0,0.0
02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.008303,0.000000,0.0,0.0,0.0,0.0
10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.222900,0.043349,0.0,0.0,0.0,0.0
100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.027678,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.045770,0.0,0.0,0.0,0.0
101,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.045265,0.000000,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
کنار,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
کنیم,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
که,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
گرفتن,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0


In [15]:
def average_cosine_similarity(tfidf):
    similarity = 0
    count = 0
    for index in range(tfidf.shape[1]):
        col = tfidf.iloc[: , index]
        for ind2 in range(index + 1, tfidf.shape[1]):
            col2 = tfidf.iloc[: , ind2]
            similarity = similarity + np.dot(col.values,col2.values)/(np.linalg.norm(col.values)*np.linalg.norm(col2.values))
            count = count + 1
    return similarity/count
            
average_cosine_similarity(tfidf)

0.11013031862360807

In [16]:
def percent_of_top_100(songs):
    count = 0
    len = songs.shape[0] #num of rows
    
    for index, row in songs.iterrows():
        if ((top100['Title'] == row['Title']) & (top100['Artist'] == row['Artist'])).any():
            count += 1
            
    return count / 100
            
display(percent_of_top_100(rock))
display(percent_of_top_100(pop))
display(percent_of_top_100(country))

0.0

0.28

0.15

In [17]:
def get_lyric_len(chart):
    return create_term_doc_matrix(chart).sum(axis = 1, skipna = True)

display(get_lyric_len(top100))

Title
Circles              300
Someone You Loved    303
Good As Hell         385
Memories             436
Roxanne              371
                    ... 
Hot Girl Summer      296
Lonely Child         573
Into The Unknown     275
Show Yourself        281
Homesick             264
Length: 99, dtype: int64

In [19]:
def get_chart_word_count(chart):
    return create_term_doc_matrix(chart).sum(axis = 0, skipna = True)

def find_most_freq_words(chart):
    counts = get_chart_word_count(chart)
    return counts.nlargest(10) 

    
freq = find_most_freq_words(top100)
display(freq)
display(freq / get_lyric_len(top100).sum(axis = 0, skipna = True))

you     1398
the     1171
me       839
it       724
and      617
to       607
my       568
yeah     471
that     450
on       396
dtype: int64

you     0.036438
the     0.030521
me      0.021868
it      0.018870
and     0.016082
to      0.015821
my      0.014804
yeah    0.012276
that    0.011729
on      0.010321
dtype: float64

In [23]:
def print_word_freq(chart):
    freq = find_most_freq_words(chart)
    
    display(freq)
    display(freq / get_lyric_len(chart).sum(axis = 0, skipna = True))

In [24]:
rock = populate_chart_with_lyrics(rock)
print_word_freq(rock)

you    637
the    550
me     322
to     309
it     255
and    249
my     226
oh     195
be     183
don    157
dtype: int64

you    0.044986
the    0.038842
me     0.022740
to     0.021822
it     0.018008
and    0.017585
my     0.015960
oh     0.013771
be     0.012924
don    0.011088
dtype: float64

In [25]:
pop = populate_chart_with_lyrics(pop)
print_word_freq(pop)

you     644
the     422
me      309
to      302
it      277
and     260
my      228
that    190
all     169
oh      169
dtype: int64

you     0.044912
the     0.029430
me      0.021550
to      0.021061
it      0.019318
and     0.018132
my      0.015901
that    0.013251
all     0.011786
oh      0.011786
dtype: float64

In [26]:
country = populate_chart_with_lyrics(country)
print_word_freq(country)

the    586
you    564
and    382
to     291
it     272
in     200
on     194
we     186
my     182
of     181
dtype: int64

the    0.039883
you    0.038386
and    0.025999
to     0.019805
it     0.018512
in     0.013612
on     0.013204
we     0.012659
my     0.012387
of     0.012319
dtype: float64

In [30]:
def find_most_freq_key_words(chart): #This removes unimportant words such as a, an, to, the, etc... 
    words = ['a', 'an', 'the', 'of', 'it', 'that', 'to', 'on', 'and', 'oh', 'be', 'in', 'when', 're', 'but', 'can', 'for', 'so']
    counts = get_chart_word_count(chart)
    counts = counts[~counts.index.isin(words)]
    return counts.nlargest(10) 

In [31]:
def print_word_key_freq(chart):
    freq = find_most_freq_key_words(chart)
    
    display(freq)
    display(freq / get_lyric_len(chart).sum(axis = 0, skipna = True))

In [32]:
print_word_key_freq(rock)

you      637
me       322
my       226
don      157
know     140
yeah     126
we       118
up       115
never    106
ll        98
dtype: int64

you      0.044986
me       0.022740
my       0.015960
don      0.011088
know     0.009887
yeah     0.008898
we       0.008333
up       0.008121
never    0.007486
ll       0.006921
dtype: float64

In [33]:
print_word_key_freq(pop)

you     644
me      309
my      228
all     169
we      156
yeah    156
don     145
love    124
your    120
la      101
dtype: int64

you     0.044912
me      0.021550
my      0.015901
all     0.011786
we      0.010879
yeah    0.010879
don     0.010112
love    0.008648
your    0.008369
la      0.007044
dtype: float64

In [34]:
print_word_key_freq(country)

you     564
we      186
my      182
me      169
your    156
got     149
what    120
if      119
all     115
one     108
dtype: int64

you     0.038386
we      0.012659
my      0.012387
me      0.011502
your    0.010617
got     0.010141
what    0.008167
if      0.008099
all     0.007827
one     0.007350
dtype: float64

In [36]:
print_word_key_freq(top100)

you     1398
me       839
my       568
yeah     471
we       353
don      317
like     284
all      277
your     274
got      268
dtype: int64

you     0.036438
me      0.021868
my      0.014804
yeah    0.012276
we      0.009201
don     0.008262
like    0.007402
all     0.007220
your    0.007142
got     0.006985
dtype: float64

In [39]:
#Are songs becoming more profane over time?
#We test this by looking into the percentage of lyrics that are profanities

!pip install profanity-check

from profanity_check import predict, predict_prob

array([1])

In [68]:
def get_profan_freq(chart):
    freq = get_chart_word_count(chart)
    profan = {}
    percent = {}
    
    for row in freq.index:
        if predict([row]):
            profan[row] = freq[row]
            percent[row] = profan[row] / get_lyric_len(chart).sum(axis = 0, skipna = True)
               
    display(profan)
    display(percent)

In [69]:
get_profan_freq(top100)

{'ass': 30,
 'asshole': 3,
 'bastard': 1,
 'bitch': 102,
 'bitches': 20,
 'bullshit': 2,
 'cum': 1,
 'dick': 15,
 'fuck': 93,
 'fucked': 3,
 'fuckin': 10,
 'fucking': 8,
 'hoe': 1,
 'hoes': 10,
 'motherfucker': 2,
 'nigga': 95,
 'pussy': 14,
 'shit': 50,
 'stupid': 11}

{'ass': 0.000781921964187974,
 'asshole': 7.81921964187974e-05,
 'bastard': 2.6064065472932468e-05,
 'bitch': 0.0026585346782391117,
 'bitches': 0.0005212813094586493,
 'bullshit': 5.2128130945864936e-05,
 'cum': 2.6064065472932468e-05,
 'dick': 0.000390960982093987,
 'fuck': 0.0024239580889827196,
 'fucked': 7.81921964187974e-05,
 'fuckin': 0.00026064065472932466,
 'fucking': 0.00020851252378345975,
 'hoe': 2.6064065472932468e-05,
 'hoes': 0.00026064065472932466,
 'motherfucker': 5.2128130945864936e-05,
 'nigga': 0.0024760862199285846,
 'pussy': 0.00036489691662105453,
 'shit': 0.0013032032736466233,
 'stupid': 0.00028670472020225714}

In [70]:
get_profan_freq(pop)

{'ass': 7,
 'asshole': 3,
 'bitch': 21,
 'bitches': 4,
 'bullshit': 1,
 'dick': 5,
 'fuck': 25,
 'fucked': 2,
 'fuckin': 1,
 'hoes': 5,
 'nigga': 20,
 'pussy': 5,
 'shit': 15,
 'stupid': 6,
 'suck': 1}

{'ass': 0.0004881790919868889,
 'asshole': 0.0002092196108515238,
 'bitch': 0.0014645372759606668,
 'bitches': 0.00027895948113536507,
 'bullshit': 6.973987028384127e-05,
 'dick': 0.00034869935141920635,
 'fuck': 0.001743496757096032,
 'fucked': 0.00013947974056768253,
 'fuckin': 6.973987028384127e-05,
 'hoes': 0.00034869935141920635,
 'nigga': 0.0013947974056768254,
 'pussy': 0.00034869935141920635,
 'shit': 0.001046098054257619,
 'stupid': 0.0004184392217030476,
 'suck': 6.973987028384127e-05}

In [89]:
#Matching songs based off of lyrics
#A person enters n lyrics and we try to match which song in top100 is the best match
def find_best_match(words):
    print(tfidf[tfidf.index.isin(words)].sum(axis = 0, skipna = True).nlargest(1))

num = int(input("How many key words do you want to enter? "))
words = []

for i in range(num - 1):
    words.append(input("Please enter a key word: "))
words.append(input("Please enter your final key word: "))  

find_best_match(words)

How many key words do you want to enter? 4
Please enter a key word: men
Please enter a key word: great
Please enter a key word: truth
Please enter your final key word: hurts
Title
Truth Hurts    0.573256
dtype: float64
